Hello everybody.

# Libraries and Data import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import datatable as dt  # pip install datatable
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

import random

from matplotlib.lines import Line2D

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve


import optuna

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# import warnings
# warnings.simplefilter(action='ignore', category=UserWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('../input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
# Read the data
train = dt.fread("../input/tabular-playground-series-sep-2021/train.csv").to_pandas().set_index("id")
test = dt.fread("../input/tabular-playground-series-sep-2021/test.csv").to_pandas().set_index("id")


# Memory reducing
taken from: https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train = reduce_memory_usage(train, verbose=True)
test = reduce_memory_usage(test, verbose=True)

In [ ]:
print("(train, test) na --> ",(train.isna().sum().sum(), test.isna().sum().sum()))

In [ ]:
is_na_train_df = train.drop(columns="claim").isna().sum(axis = 1)
print(is_na_train_df.shape)

is_na_test_df = test.isna().sum(axis = 1)
print(is_na_test_df.shape)

In [ ]:
is_na_train_sum = train.drop(columns="claim").isna().sum(axis=1)
print(is_na_train_sum.shape)

is_na_test_sum = test.isna().sum(axis=1)
print(is_na_test_sum.shape)

In [ ]:
# from https://www.kaggle.com/sgiuri/sep21tp-na-feature-importance 
na_fi = ['f74', 'f91', 'f107', 'f53', 'f7', 'f12', 'f48', 'f19', 'f100', 'f69', 'f18', 'f112', 'f66', 'f17', 'f113']
is_na_train_df = train.drop(columns="claim").isna().loc[:,na_fi].astype(int)
print(is_na_train_df.shape)

is_na_test_df = test.isna().loc[:,na_fi].astype(int)
print(is_na_test_df.shape)

In [ ]:
train = train.join(is_na_train_df, rsuffix='_isNa')
test = test.join(is_na_test_df, rsuffix='_isNa')

In [ ]:
train["isNA"] =is_na_train_sum
print(train.shape)
test["isNA"] = is_na_test_sum
print(test.shape)

# train = train.head(1000)
# test = test.head(1000)
# if len(train == 1000):
#     print("Train test reduced to increase speed in try and test ")
    

## Data preparation: Siple Imputer + NA to median

In [ ]:
x_Mm_scaler = MinMaxScaler()
X = pd.DataFrame(x_Mm_scaler.fit_transform(train.drop("claim", axis=1)),
                 columns=train.drop("claim", axis=1).columns)
y = train.claim.astype(int)
X_test = pd.DataFrame(x_Mm_scaler.transform(test), columns=test.columns)

In [ ]:
imputer_zeros = SimpleImputer(strategy="median")
X = pd.DataFrame(imputer_zeros.fit_transform(train.drop("claim", axis=1)),
                 columns=train.drop("claim", axis=1).columns)
X_test = pd.DataFrame(imputer_zeros.transform(test), columns=test.columns)
X = pd.DataFrame(x_Mm_scaler.fit_transform(X),
                 columns=train.drop("claim", axis=1).columns)
X_test = pd.DataFrame(x_Mm_scaler.transform(X_test), columns=test.columns)
print("(train, test) na --> ",(X.isna().sum().sum(), X_test.isna().sum().sum()))

In [ ]:
# X = reduce_memory_usage(X, verbose=True)
# X_test = reduce_memory_usage(X_test, verbose=True)

In [ ]:
def train_model_optuna_xgb(trial, X_train, X_valid, y_train, y_valid):
    """
    A function to train a model using different hyperparamerters combinations provided by Optuna. 
    Loss of validation data predictions is returned to estimate hyperparameters effectiveness.
    """
    preds = 0
       
    #A set of hyperparameters to optimize by optuna
    xgb_params = {
                 "n_estimators": trial.suggest_categorical('n_estimators', [10000]),
                 "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.8),
                 "subsample": trial.suggest_float('subsample', 0.5, 0.95),
                 "colsample_bytree": trial.suggest_float('colsample_bytree', 0.5, 0.95),
                 "max_depth": trial.suggest_int("max_depth", 5, 16),
                 "booster": trial.suggest_categorical('booster', ["gbtree"]),
                 "tree_method": trial.suggest_categorical('tree_method', ["gpu_hist"]),
                 "reg_lambda": trial.suggest_float('reg_lambda', 2, 100),
                 "reg_alpha": trial.suggest_float('reg_alpha', 1, 50),
                 "random_state": trial.suggest_categorical('random_state', [42]),
                 "n_jobs": trial.suggest_categorical('n_jobs', [4]),
                    }

    # Model loading and training
    model = XGBClassifier(**xgb_params)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="rmse",
              early_stopping_rounds=100,
              verbose=False)
    
    print(f"Number of boosting rounds: {model.best_iteration}")
    oof = model.predict(X_valid)
    oof[oof<0] = 0
    
    return np.sqrt(mean_squared_error(y_valid, oof))

In [ ]:
%%time
n_trials = 200

skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
if len(train)== 1000:
    n_trials = 2

for fold, (train_indicies, valid_indicies) in enumerate(skf.split(X,y)):
    
    X_train, X_valid = X.loc[train_indicies], X.loc[valid_indicies]
    y_train, y_valid = y.loc[train_indicies], y.loc[valid_indicies]

# Setting optuna verbosity to show only warning messages
# If the line is uncommeted each iteration results will be shown
optuna.logging.set_verbosity(optuna.logging.WARNING)
time_limit = 3600 * 3
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: train_model_optuna_xgb(trial, 
                                                X_train, 
                                                X_valid,
                                                y_train, 
                                                y_valid),
               n_trials = n_trials,
               timeout=time_limit
              )
 # Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)


In [ ]:
xgb_params = study.best_params
# xgb_params = {'n_estimators': 10000, 
#               'learning_rate': 0.08625196792060146, 
#               'subsample': 0.5959773829663169, 
#               'colsample_bytree': 0.7603045913120982, 
#               'max_depth': 7, 'booster': 'gbtree', 
#               'tree_method': 'gpu_hist', 
#               'reg_lambda': 74.60593770387143, 
#               'reg_alpha': 33.38858560681472, 
#               'random_state': 42, 
#               'n_jobs': 4}

In [ ]:
def strati_fit(X, y, X_test, 
               splits=6, random_state=42,
               model = XGBClassifier(**xgb_params)):
    
    splits = splits
    skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros((X.shape[0],))
    preds = 0
    train_preds = 0
    model_fi = 0
    total_mean_rmse = 0
    total_mean_roc_auc_score = 0
    
    
    for fold, (train_indicies, valid_indicies) in enumerate(skf.split(X,y)):

        X_train, X_valid = X.loc[train_indicies], X.loc[valid_indicies]
        y_train, y_valid = y.loc[train_indicies], y.loc[valid_indicies]
        print(fold, f"X_train = {X_train.shape} - y_train: {y_train.shape}")
        print(fold, f"X_valid = {X_valid.shape} - y_valid: {y_valid.shape}")
        
        model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=100,
              verbose=False)
        # print("fitted")
        preds += (model.predict_proba(X_test))[:,1] / splits
        train_preds += (model.predict_proba(X))[:,1] / splits
        print(train_preds.shape)
        # print("preds ok")
        model_fi += model.feature_importances_
        # print("model_fi ok")
        oof_preds[valid_indicies] = model.predict_proba(X_valid)[:,1]
        oof_preds[oof_preds < 0] = 0
    #     fold_rmse = np.sqrt(mean_squared_error(y_scaler.inverse_transform(np.array(y_valid).reshape(-1,1)), y_scaler.inverse_transform(np.array(oof_preds[valid_idx]).reshape(-1,1))))
        fold_rmse = np.sqrt(mean_squared_error(y_valid, oof_preds[valid_indicies]))
        fold_roc_auc_score = roc_auc_score(y_valid, oof_preds[valid_indicies])
        
        print(f"/nFold {fold} ROC AUC Score: {fold_roc_auc_score}")
        
        print(f"Fold {fold} RMSE: {fold_rmse}")
        total_mean_rmse += fold_rmse / splits
        total_mean_roc_auc_score += fold_roc_auc_score / splits
    return preds, model_fi, total_mean_rmse, total_mean_roc_auc_score, train_preds
    

In [ ]:
random_states = [0,3,42,69,666]
predictions = pd.DataFrame()
predictions["id"] = test.index
train_predictions = pd.DataFrame()
train_predictions["id"] = X.index

study = pd.DataFrame()

for random_state in random_states:
    
    preds, model_fi, total_mean_rmse, total_mean_roc_auc_score, train_preds = strati_fit(X, y, X_test, 
           splits=10, random_state=random_state, model = XGBClassifier(**xgb_params))
    predictions[random_state] = preds
    train_predictions[random_state] = train_preds
    study[random_state] = model_fi, total_mean_rmse, total_mean_roc_auc_score


In [ ]:
study.to_csv('study.csv', index=False, header=study.columns)
study.head()

In [ ]:
train_predictions.to_csv('train_predictions.csv', index=False, header=train_predictions.columns)
train_predictions.head()

In [ ]:
predictions.to_csv('submission_pre_ensamble.csv', index=False, header=predictions.columns)
predictions.head()